In [8]:
import numpy as np
import pandas as pd

# User defines (can be set as default arguments or passed in)
user_core_density = 7874.0  # kg/m^3
user_mantle_density = 3000.0  # kg/m^3

In [9]:
col_names = ["t", "type", "new_id", "parent1_id", "parent2_id", "new_mass", "parent1_mass", "parent2_mass", "new_radius",
             "parent1_radius", "parent2_radius", "v_imp", "theta_imp"]

fam_tree = pd.read_csv("family_tree.csv", names=col_names)
fam_tree['cmf'] = np.nan

fam_tree.head()

,t,type,new_id,parent1_id,parent2_id,new_mass,parent1_mass,parent2_mass,new_radius,parent1_radius,parent2_radius,v_imp,theta_imp,cmf
0,532.3162,2,157,3,4,5.600000e-07,2.800000e-07,2.800000e-07,0.000030,0.000024,0.000024,0.987347,0.669639,NaN
1,572.3846,2,159,9,98,3.080000e-07,2.800000e-07,2.800000e-08,0.000024,0.000024,0.000011,0.857217,1.045487,NaN
2,2707.0600,2,161,95,8,3.080000e-07,2.800000e-08,2.800000e-07,0.000024,0.000011,0.000024,0.840215,1.188651,NaN
3,8714.3880,3,163,87,7,3.080000e-07,2.800000e-08,2.800000e-07,0.000024,0.000011,0.000024,0.861587,0.729169,NaN
4,10552.1600,2,165,47,50,5.600000e-08,2.800000e-08,2.800000e-08,0.000014,0.000011,0.000011,0.458682,0.857211,NaN


In [10]:
merge_types = [1, 2, 3]
erosion_types = [4, 5, 7, 10]
elastic_bounce_types = [6, 8, 9]

# Tracking Core Mass Fraction (CMF) of Bodies

To track the particles CMFs, we can divide the collision types into three categories: merging, bouncing, and erosion/accretion. 

## Merging

When two particles merge, we assume that their cores mix and their CMF will be the average of the parent CMFs.

$CMF_{new} = (CMF_{parent1}M_{parent1} + CMF_{parent2}M_{parent2})/(M_{parent1} + M_{parent2})$

Where M_{parent1}, M_{parent2}, CMF_{parent1}, CMF_{parent2} are the masses of parent 1 and 2, and CMF of parent 1 and 2 respectively. 

## Bouncing

When two particles go through an elastic (or inelastic) bounce, their masses and CMFs remain unchanged.

## Erosion/Accretion

There are multiple options for how to compute the CMFs of objects after an erosion or accretion event. Here, we demonstrate two examples from Marcus et al. (2010). 

### Model 1

In the first model we assume that the cores of the two bodies merge, and the escaping mass will be made from the lightest possible material. 

We will have to find the largest remnant (denoted with lr) and we will compute a CMF for the largest remnant, and a CMF for fragments. We assume that all fragments will have the same CMF. 

The largest remnant core mass ($M_{core, lr}$) will be:

$M_{core, lr} = min(M_{lr}, M_{core, avail})$

Where $ M_{core, avail} = CMF_{parent1} M_{parent1} + CMF_{parent2}  M_{parent2} $. Then, CMFs are: 

$CMF_{lr} = M_{core, lr}/M_{lr}$

$CMF_{frag} = (M_{core, avail} - M_{core, lr})/(M_{tot} - M_{lr})$

Where $M_{tot}$ is the total mass ($M_{tot} = M_{parent1} + M_{parent2}$)

### Model 2

In the second model, we first need to identify target and projectile, and see if the collision has resulted in an accretion ($Mlr > M_{target}$) or erosion ($Mlr > M_{target}$). 

In the accretion regime, core material from the projectile will be accreted into target first, followed by mantle material. Therefore, the mass of the largest remnant's core will be:

$M_{core, lr} = M_{core, target} + min(M_{core, avail} - M_{core, target}, M_{lr} - M_{target})$

In the erosion regime, mantle material is ejected from the target untill there is no more left, and then core material will be ejected:

$M_{core, lr} = min(M_{core, target}, M_{lr})$

And in all cases, CMFs will be derived as before:

$CMF_{lr} = M_{core, lr} / M_{lr}$

$CMF_{frags} = (M_{core, avail} - M_{core, lr})/(M_{tot} - M_{lr})$

In [11]:
# Choose erosion/accretion model here
# 1 for Model 1, and 2 for Model 2
erosion_accretion_model = 1

cmf_i = 0.3
tolerance = 1e-8

processed = set()  # (parent1_id, parent2_id) already handled


def get_parent_cmf(fam_tree, parent_id, cmf_i):
    parent = fam_tree.loc[fam_tree['new_id'] == parent_id]
    if parent.empty or pd.isna(parent['cmf'].iloc[0]):
        return cmf_i
    return parent['cmf'].iloc[0]


for idx, row in fam_tree.iterrows():

    key = (row['parent1_id'], row['parent2_id'])

    # Skip if this collision already processed
    if key in processed:
        continue
    processed.add(key)

    # All children of this collision (siblings)
    group = fam_tree[
        (fam_tree['parent1_id'] == key[0]) &
        (fam_tree['parent2_id'] == key[1])
    ]

    parent1_cmf = get_parent_cmf(fam_tree, key[0], cmf_i)
    parent2_cmf = get_parent_cmf(fam_tree, key[1], cmf_i)

    collision_type = group['type'].iloc[0]

    # =======================
    # ELASTIC BOUNCE
    # =======================
    if collision_type in elastic_bounce_types:
        for i, r in group.iterrows():
            if abs(r['new_mass'] - r['parent1_mass']) < tolerance:
                fam_tree.loc[i, 'cmf'] = parent1_cmf
            else:
                fam_tree.loc[i, 'cmf'] = parent2_cmf

    # =======================
    # MERGE
    # =======================
    elif collision_type in merge_types:
        for i, r in group.iterrows():
            fam_tree.loc[i, 'cmf'] = (r['parent1_mass'] * parent1_cmf + r['parent2_mass'] * parent2_cmf) / (r['parent1_mass'] + r['parent2_mass'])

    # =======================
    # EROSION AND ACCRETION
    # =======================
    elif collision_type in erosion_types:
        idx_max = group['new_mass'].idxmax()
        Mlr = group.loc[idx_max, 'new_mass']
        M_tot = group['parent1_mass'].iloc[0] + group['parent2_mass'].iloc[0]
        M_core_tot = (
            parent1_cmf * group['parent1_mass'].iloc[0] +
            parent2_cmf * group['parent2_mass'].iloc[0]
        )
        if erosion_accretion_model == 1:
            M_core_lr = min(Mlr, M_core_tot)

        if erosion_accretion_model == 2:
            if (group['parent1_mass'].iloc[0] >= group['parent2_mass'].iloc[0]):
                M_target = group['parent1_mass'].iloc[0]
                M_core_target = M_target * parent1_cmf
            else:
                M_target = group['parent2_mass'].iloc[0]
                M_core_target = M_target * parent2_cmf

            sum_M_frags = group['new_mass'].sum() - Mlr

            # Accretion
            if Mlr >= M_target:
                M_core_lr = M_core_target + min(M_core_tot - M_core_target, Mlr - M_target)
            # Erosion
            else:
                M_core_lr = min(M_core_target, Mlr)
 
        CMF_lr = abs(M_core_lr/Mlr)
        CMF_frag = abs((M_core_tot - M_core_lr)/(M_tot - Mlr))

        fam_tree.loc[idx_max, 'cmf'] = CMF_lr
        for i in group.index:
            if i != idx_max:
                fam_tree.loc[i, 'cmf'] = CMF_frag


We can look at erosions as an example:

In [12]:
erosions = fam_tree[fam_tree['type'].isin(erosion_types)]
erosions.head()

,t,type,new_id,parent1_id,parent2_id,new_mass,parent1_mass,parent2_mass,new_radius,parent1_radius,parent2_radius,v_imp,theta_imp,cmf
19,154115.8,4,190,1,183,5.183492e-07,2.800000e-07,2.800000e-07,0.000029,0.000024,0.000024,1.079249,0.513457,0.324106
20,154115.8,4,191,1,183,4.165083e-08,2.800000e-07,2.800000e-07,0.000013,0.000024,0.000024,1.079249,0.513457,0.000000
29,342803.5,5,208,56,174,1.400000e-08,2.800000e-08,2.800000e-08,0.000009,0.000011,0.000011,1.724715,0.358070,0.057016
30,342803.5,5,209,56,174,1.442982e-08,2.800000e-08,2.800000e-08,0.000009,0.000011,0.000011,1.724715,0.358070,1.000000
31,342803.5,5,210,56,174,1.391219e-08,2.800000e-08,2.800000e-08,0.000009,0.000011,0.000011,1.724715,0.358070,0.057016
